In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB1  # Updated to B1 variant
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# ========================
# Configuration
# ========================
SEED = 42
IMG_SIZE = (240, 240)  # Increased input size
BATCH_SIZE = 32
EPOCHS = 50  # Increased epochs with early stopping
NUM_CLASSES = 5
INIT_LR = 1e-4  # Reduced initial learning rate
WEIGHT_DECAY = 1e-4

In [3]:

# ========================
# Enhanced Data Pipeline
# ========================

def build_augmentation():
    return keras.Sequential([
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.25),
        layers.RandomZoom(0.25),
        layers.RandomContrast(0.25),
        layers.RandomBrightness(0.2),
        layers.GaussianNoise(0.1),
        layers.RandomTranslation(0.15, 0.15),
        layers.RandomShear(0.2),
    ], name="augmentation")

def create_datagen(augmentation):
    return ImageDataGenerator(
        preprocessing_function=preprocess_input,  # EfficientNet's custom preprocessing
        validation_split=0.2  # Using validation split
    )


In [9]:
train_datagen = create_datagen(build_augmentation())
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=SEED,
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    seed=SEED,
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

AttributeError: module 'keras._tf_keras.keras.layers' has no attribute 'RandomShear'

In [ ]:
# ========================
# Enhanced Model Architecture
# ========================

def create_model():
    # Load base model with pretrained weights
    base_model = EfficientNetB1(
        include_top=False,
        weights='imagenet',
        input_shape=IMG_SIZE + (3,),
        pooling='avg'
    )
    
    # Freeze initial layers
    for layer in base_model.layers[:150]:
        layer.trainable = False
        
    # Custom head with regularization
    inputs = keras.Input(shape=IMG_SIZE + (3,))
    x = build_augmentation()(inputs)  # Built-in augmentation
    x = base_model(x)
    x = layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(WEIGHT_DECAY))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(WEIGHT_DECAY))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    
    # Unfreeze more layers after initial training
    return model

model = create_model()
